In [24]:
import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, punkt
from gensim.models import KeyedVectors

#setting up libraries
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
import spacy


In [25]:
df = pd.read_pickle("/home/seh6fy/git/publicrd/data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")


In [26]:
df = df[~df.DEPARTMENT.isnull()]

In [27]:
df_nsf = df.loc[df['DEPARTMENT'] == 'HHS']
df_nsf = df_nsf.iloc[0:1000,]

In [28]:
df_nasa = df.loc[df['DEPARTMENT'] == 'NASA']
df_nasa = df_nasa.iloc[0:1000,]

In [38]:
frames = [df_nsf, df_nasa]
df_split = pd.concat(frames)
df_split.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens
13120,150681,DESCRIPTION (provided by applicant): The use ...,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...
13121,150683,DESCRIPTION (provided by applicant): The patho...,5-(6)-carboxyfluorescein diacetate succinimidy...,"DOUBLE NEGATIVE, CD4-CD8- T CELLS IN HUMAN LEI...",HHS,NIH,NIAID,1R03AI066253-01A1,9/30/2008,9/29/2011,"GOLLOB, KENNETH J",NaN,NaN,899644116,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BELO HORIZONTE,NaN,31270-901,BRAZIL,9/30/2008,9/29/2009,93.855,2008,54000.0,NaN,1,1,54000.0,1,pathogen leishmania protozoan_parasite cause w...
13122,150684,DESCRIPTION (provided by applicant): \t In thi...,Adopted; AIDS clinical trial group; AIDS/HIV p...,SAO PAULO CLINICAL TRIALS UNIT,HHS,NIH,NIAID,1U01AI069420-01,1/1/2008,12/31/2013,"KALLAS, ESPER GEORGES",NaN,NaN,914608039,FEDERAL UNIVERSITY OF SAO PAULO,SAO PAULO,NaN,04037-003,BRAZIL,1/1/2008,12/31/2008,93.855,2008,493116.0,NaN,1,1,493116.0,1,application propose constitution_sao_paulo cli...
13123,150685,DESCRIPTION (provided by applicant): \t The O...,AIDS clinical trial group; AIDS prevention; ba...,THE FIOCRUZ THERAPEUTIC AND PREVENTION HIV/AID...,HHS,NIH,NIAID,1U01AI069476-01,1/15/2008,12/31/2013,"GRINSZTEJN, BEATRIZ",NaN,NaN,899294177,FUNDACAO OSWALDO CRUZ,RIO DE JANEIRO,NaN,21040-360,BRAZIL,1/15/2008,12/31/2008,93.855,2008,3194471.0,NaN,1,1,3194471.0,1,oswaldo_cruz foundation fiocruz therapeutic pr...
13124,150686,DESCRIPTION (provided by applicant): \t Fe...,AIDS prevention; AIDS/HIV problem; American; b...,CLINICAL TRIAL UNIT FOR HIV PREVENTION AND MIC...,HHS,NIH,NIAID,1U01AI069480-01,9/8/2008,11/30/2013,"MAYER, KENNETH H",NaN,07,072366156,FENWAY COMMUNITY HEALTH CENTER,BOSTON,MA,022154302,UNITED STATES,9/8/2008,11/30/2008,93.855,2008,637625.0,NaN,1,1,637625.0,1,fenway community health fch_boston community h...


In [39]:
df_split= df_split.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [40]:
s = df_split["ABSTRACT"].str.split('.').apply(pd.Series,1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'ABSTRACT'

In [41]:
s.replace('', np.nan, inplace=True)
s.dropna(inplace=True)

In [42]:
del df_split['ABSTRACT']
df_split = df_split.join(s)
df_split.head(10)

,index,PROJECT_ID,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens,ABSTRACT
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,DESCRIPTION (provided by applicant): The use ...
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,MDMA is a potent serotonin (5-HT) releaser
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,"In animals, exposure to MDMA is associated wi..."
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,As doses used in humans overlap with doses kn...
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,5-HT neurotoxicity might lead to neurocogniti...
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,Recent progress in nuclear medicine has e...
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,The 5-HT transporter (SERT) is located exclus...
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONIN TRANSMISSION IN MDMA ...,HHS,NIH,NIDA,5R01DA015806-04,8/15/2003,4/30/2010,"ABI-DARGHAM, ANISSA",NaN,13,167204994,NEW YORK STATE PSYCHIATRIC INSTITUTE,NEW YORK,NY,100321007,UNITED STATES,5/1/2008,4/30/2010,93.279,2008,409348.0,NaN,1,1,409348.0,1,use recreational drug methylenedioxymethamphet...,Recent imaging studies of the SERT with PET a...
0,13120,150681,Abstinence; Adult; Animals; Anxiety Disorders;...,PET IMAGING OF SEROTONI

In [43]:
from gensim.models import Word2Vec

word2vec = Word2Vec(df_split['ABSTRACT'], min_count=10)

KeyError: "Key 'research' not present"

In [20]:
df_split = df_split.explode("ABSTRACT", ignore_index=True)
df_split.rename(columns={"Unnamed: 0": "Index ID"}, inplace=True)
df_split.index.name = "Sentence ID"

In [22]:
df_split['ABSTRACT']

Sentence ID
0       DESCRIPTION (provided by applicant):  The use ...
1       DESCRIPTION (provided by applicant): The patho...
2       DESCRIPTION (provided by applicant): \t In thi...
3       DESCRIPTION (provided by applicant): \t  The O...
4       DESCRIPTION (provided by applicant): \t     Fe...
                              ...                        
1995    STEPPING STONES TOWARD CME PREDICTION: CHARACT...
1996    Spectroscopic Studies of Comets  Comets repres...
1997    THE RELATIVISTIC IRON LINE - QPO CONNECTION IN...
1998    A Consistent Evolutionary History of Jupiter a...
1999    REMOTE SENSING AND GEOLOGIC STUDIES OF PLANETA...
Name: ABSTRACT, Length: 2000, dtype: object

NameError: name 'crossvalidation_data' is not defined

In [ ]:



#just to be sure we got all the sentences
print(len(train_list_of_sent))
print(len(test_list_of_sent))
print(len(crossvalidation_list_of_sent))
w2v_vector=w2v_model.wv.vectors
w2v_vector.shape
def find_word2vec(list_of_sent):
    w2v=[]
    for sent in tqdm(list_of_sent):#for each sentence
        sent_vector=np.zeros(300)#create 300 dimensions of zeros
        for word in sent:#for each word in sentence
            if word in w2v_model.wv.vocab:#if word exists in word2vec model
                vec=w2v_model.wv[word]#get the vector representation of the word
                sent_vector+=vec#add the vector to sent_vector
        #after adding all the word vectors in the sentence add the vector that now represents 
        #the whole sentence to vectors_of_sentences
        w2v.append(sent_vector)
    return w2v
#word2vec representation of train_data
train_w2v=find_word2vec(train_list_of_sent)
crossvalidation_w2v=find_word2vec(crossvalidation_list_of_sent)
test_w2v=find_word2vec(test_list_of_sent)